# MC Truth vs. Recon study
Take a look at (the difference between) the reconstructed and truth values for final-state kinematic components.

In [4]:
import numpy as np
import pandas as pd
import uproot
import boost_histogram as bh
import functools as ft
import matplotlib.pyplot as plt
import matplotlib.colors as cols
# %matplotlib notebook
%matplotlib widget

In [2]:
plt.style.use('seaborn')
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.formatter.limits'] = (-3,3)
plt.rcParams['axes.formatter.use_mathtext'] = True
plt.rcParams['font.size']= 16
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

In [3]:
#data_root_file = "/w/work3/home/pauln/analysis/processed/chanser_ANAsub/pauln/Pi0_PID_fullcomb/FinalState.root"
#MC_root_file = "/w/work3/home/pauln/analysis/processed/chanser_MCsub/pauln/Pi0_PID_fullcomb/FinalState.root"
MC_root_file = "/home/pauln/code/pi0chanser/output/chanser_MC15_gammatruth/pauln/Pi0_PID_exclcomb/FinalState.root"

tree= "FINALOUTTREE"

# data = uproot.open(data_root_file)[tree]
# data_df = data.pandas.df()

data = uproot.open(MC_root_file)[tree]
MC_df = data.pandas.df()

In [8]:
def hist1D(ax, df, var, bins = None, cuts=None, y_err=False, color=None, lalpha=1, falpha=0.4, normed=False, filled = False, density = False, params={}, label=None):
    #select next color in colormap for use in line and fill.
    if color is None:
        color = next(ax._get_lines.prop_cycler)['color']
    
    if cuts is not None: 
        data = df[var[0]][cuts]
    else:
        data = df[var[0]]
        
    #calculate area under the curve if wanting normalised plot    
    weights = None
    if normed is True:
        entries, edges = np.histogram(
            data,
            range = (var[1][1], var[1][2]),
            bins = var[1][0]        
        )
      
        integral = sum(np.diff(edges)*entries)
        weights = np.ones_like(data)/integral
        
    if bins is None: 
        bins = var[1][0]
    
    #Draw histogram
    entries,edges,_ = ax.hist(data,
    range = (var[1][1], var[1][2]),
    bins = bins,
    density = density,
    weights=weights,
    histtype = 'step',
    linewidth = 1.5,
    alpha=lalpha,
    color = color,
    label=label,
    **params
    )
 
    #fill area under histo if set
    if filled is True:
        # brute-force: drawing it again to allow separate alpha(opacity) to be used
        h = ax.hist(data,
            range = (var[1][1], var[1][2]),
            bins = bins,
            density = density,
            weights=weights,
            histtype = 'step',
            linewidth = None,
            fill = True,
            alpha = falpha,
            color = color
            )
        
    #error bars    
    if y_err is True:
        if normed is True: 
            yerr = np.sqrt(entries)/integral
            entries = entries/integral
        else:
            entries, edges = np.histogram(
                data,
                range = (var[1][1], var[1][2]),
                bins = bins       
            )
            yerr = np.sqrt(entries)
            
        bin_width = edges[1]-edges[0]
        bin_centers = edges[:-1] + bin_width/2
        ax.errorbar(bin_centers, entries, yerr=yerr, color='r', ls='', alpha=0.6, label='err', elinewidth=0.6)
        
        
    #deal with automatic y-axis scaling funk
    # if normed is True:
    #     #axes = plt.gca()
    #     ax.set_ylim([0,max(entries/integral)*1.1])

    ax.set_xlabel(var[2])
    ax.set_xlim(var[1][1], var[1][2])

    
    return #(entries, edges, yerr)

In [5]:
#list(MC_df.columns)

In [6]:
'''
Lists through the columns in the df beginning with Pi0,
taking the difference between this variable and the corresponding
"true" value; adding this to a new column prepended "delta".
'''

for var in list(MC_df[[c for c in MC_df if c.startswith('Pi0')]].columns):
    d_Str = "delta"+var
    tru_Str = "tru"+var
    
    MC_df[d_Str] = MC_df[var] - MC_df[tru_Str]

## Particle Distributions 

Truth-Matched Distributions for each final-state particle.

In [9]:
bins = 200

e_distros = [
    ["Pi0e_px", (bins, -2, 2), r"$p_{x}$ $(GeV/c^{2})$"],
    ["Pi0e_py", (bins, -2, 2), r"$p_{y}$ $(GeV/c^{2})$"],
    ["Pi0e_pz", (bins, 0, 9), r"$p_{z}$ $(GeV/c^{2})$"],
    ["Pi0e_E", (bins, 0, 9), r"$E$ $(GeV)$"],
    ["Pi0e_magP", (bins, 0, 9), r"$|p|$ $(GeV/c^{2})$"],
    ["Pi0e_pT", (bins, 0, 2), r"$p_{T}$ $(GeV/c^{2})$"],
    ["Pi0e_theta", (bins, 0, 50), r"$\theta$ $(^{\circ})$"],
    ["Pi0e_phi", (bins, -185, 185), r"$\phi$ $(^{\circ})$"]
]

e_t_distros = [
    ["truPi0e_px", (bins, -2, 2), r"$p_{x}$ $(GeV/c^{2})$"],
    ["truPi0e_py", (bins, -2, 2), r"$p_{y}$ $(GeV/c^{2})$"],
    ["truPi0e_pz", (bins, 0, 9), r"$p_{z}$ $(GeV/c^{2})$"],
    ["truPi0e_E", (bins, 0, 9), r"$E$ $(GeV)$"],
    ["truPi0e_magP", (bins, 0, 9), r"$|p|$ $(GeV/c^{2})$"],
    ["truPi0e_pT", (bins, 0, 2), r"$p_{T}$ $(GeV/c^{2})$"],
    ["truPi0e_theta", (bins, 0, 50), r"$\theta$ $(^{\circ})$"],
    ["truPi0e_phi", (bins, -185, 185), r"$\phi$ $(^{\circ})$"]
]

In [8]:
fig1, ax1 = plt.subplots(2,4, figsize=(15,10))

ax1=ax1.flatten()

fig1.suptitle('electrons', fontsize=16)

for i, distro in enumerate(e_distros):
    hist1D(ax1[i], MC_df, distro, cuts=(MC_df.Truth==1))
    
for i, distro in enumerate(e_t_distros):
    ax=ax1[i].twinx()

    hist1D(ax, MC_df, distro, cuts=(MC_df.Truth==1), color='brown', lalpha=0.7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
bins = 120

neut_distros = [
    ["Pi0rec_px", (bins, -2, 2), r"$p_{x}$ $(GeV/c^{2})$"],
    ["Pi0rec_py", (bins, -2, 2), r"$p_{y}$ $(GeV/c^{2})$"],
    ["Pi0rec_pz", (bins, 0, 2), r"$p_{z}$ $(GeV/c^{2})$"],
    ["Pi0rec_E", (bins, 0.5, 2.5), r"$E$ $(GeV)$"],
    ["Pi0rec_magP", (bins, 0, 2), r"$|p|$ $(GeV/c^{2})$"],
    ["Pi0rec_pT", (bins, 0, 1.55), r"$p_{T}$ $(GeV/c^{2})$"],
    ["Pi0rec_theta", (bins, 0, 180), r"$\theta$ $(^{\circ})$"],
    ["Pi0rec_phi", (48, -180, 180), r"$\phi$ $(^{\circ})$"]
]

neut_t_distros = [
    ["truPi0rec_px", (bins, -2, 2), r"$p_{x}$ $(GeV/c^{2})$"],
    ["truPi0rec_py", (bins, -2, 2), r"$p_{y}$ $(GeV/c^{2})$"],
    ["truPi0rec_pz", (bins, 0, 2), r"$p_{z}$ $(GeV/c^{2})$"],
    ["truPi0rec_E", (bins, 0.5, 2.5), r"$E$ $(GeV)$"],
    ["truPi0rec_magP", (bins, 0, 2), r"$|p|$ $(GeV/c^{2})$"],
    ["truPi0rec_pT", (bins, 0, 1.55), r"$p_{T}$ $(GeV/c^{2})$"],
    ["truPi0rec_theta", (bins, 0, 120), r"$\theta$ $(^{\circ})$"],
    ["truPi0rec_phi", (48, -120, 180), r"$\phi$ $(^{\circ})$"]
]

In [10]:
fig2, ax2 = plt.subplots(2,4, figsize=(15,10))
ax2=ax2.flatten()

fig2.suptitle('neutrons', fontsize=16)

for i, distro in enumerate(neut_distros):
    hist1D(ax2[i], MC_df, distro, cuts=(MC_df.Truth==1))
    
for i, distro in enumerate(neut_t_distros):
    ax=ax2[i].twinx()

    hist1D(ax, MC_df, distro, cuts=(MC_df.Truth==1), color='brown', lalpha=0.7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
bins = 200

phot1_distros = [
    ["Pi0phot1_px", (bins, -1.75, 1.75), r"$p_{x}$ $(GeV/c^{2})$"],
    ["Pi0phot1_py", (bins, -1.75, 1.75), r"$p_{y}$ $(GeV/c^{2})$"],
    ["Pi0phot1_pz", (bins, 0, 8), r"$p_{z}$ $(GeV/c^{2})$"],
    ["Pi0phot1_E", (bins, 0, 8), r"$ E$ $(GeV)$"],
    ["Pi0phot1_magP", (bins, 0, 8), r"$|p|$ $(GeV/c^{2})$"],
    ["Pi0phot1_pT", (bins, 0, 2), r"$p_{T}$ $(GeV/c^{2})$"],
    ["Pi0phot1_theta", (bins, 0, 40), r"$\theta$ $(^{\circ})$"],
    ["Pi0phot1_phi", (bins, -180, 180), r"$\phi$ $(^{\circ})$"]
]

phot1_t_distros = [
    ["truPi0phot1_px", (bins, -1.75, 1.75), r"$p_{x}$ $(GeV/c^{2})$"],
    ["truPi0phot1_py", (bins, -1.75, 1.75), r"$p_{y}$ $(GeV/c^{2})$"],
    ["truPi0phot1_pz", (bins, 0, 8), r"$p_{z}$ $(GeV/c^{2})$"],
    ["truPi0phot1_E", (bins, 0, 8), r"$ E$ $(GeV)$"],
    ["truPi0phot1_magP", (bins, 0, 8), r"$|p|$ $(GeV/c^{2})$"],
    ["truPi0phot1_pT", (bins, 0, 2), r"$p_{T}$ $(GeV/c^{2})$"],
    ["truPi0phot1_theta", (bins, 0, 40), r"$\theta$ $(^{\circ})$"],
    ["truPi0phot1_phi", (bins, -180, 180), r"$\phi$ $(^{\circ})$"]
]

In [12]:
fig3, ax3 = plt.subplots(2,4, figsize=(15,10))
ax3=ax3.flatten()

fig3.suptitle('photons (1)', fontsize=16)

for i, distro in enumerate(phot1_distros):
    hist1D(ax3[i], MC_df, distro, cuts=(MC_df.Truth==1))
    
for i, distro in enumerate(phot1_t_distros):
    ax=ax3[i].twinx()

    hist1D(ax, MC_df, distro, cuts=(MC_df.Truth==1), color='brown', lalpha=0.7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
bins = 200

phot2_distros = [
    ["Pi0phot2_px", (bins, -1, 1), r"$p_{x}$ $(GeV/c^{2})$"],
    ["Pi0phot2_py", (bins, -1, 1), r"$p_{y}$ $(GeV/c^{2})$"],
    ["Pi0phot2_pz", (bins, 0, 4), r"$p_{z}$ $(GeV/c^{2})$"],
    ["Pi0phot2_E", (bins, 0, 4), r"$ E$ $(GeV)$"],
    ["Pi0phot2_magP", (bins, 0, 4), r"$|p|$ $(GeV/c^{2})$"],
    ["Pi0phot2_pT", (bins, 0, 1), r"$p_{T}$ $(GeV/c^{2})$"],
    ["Pi0phot2_theta", (bins, 0, 40), r"$\theta$ $(^{\circ})$"],
    ["Pi0phot2_phi", (bins, -180, 180), r"$\phi$ $(^{\circ})$"]
]

phot2_t_distros = [
    ["truPi0phot2_px", (bins, -1, 1), r"$p_{x}$ $(GeV/c^{2})$"],
    ["truPi0phot2_py", (bins, -1, 1), r"$p_{y}$ $(GeV/c^{2})$"],
    ["truPi0phot2_pz", (bins, 0, 4), r"$p_{z}$ $(GeV/c^{2})$"],
    ["truPi0phot2_E", (bins, 0, 4), r"$ E$ $(GeV)$"],
    ["truPi0phot2_magP", (bins, 0, 4), r"$|p|$ $(GeV/c^{2})$"],
    ["truPi0phot2_pT", (bins, 0, 1), r"$p_{T}$ $(GeV/c^{2})$"],
    ["truPi0phot2_theta", (bins, 0, 40), r"$\theta$ $(^{\circ})$"],
    ["truPi0phot2_phi", (bins, -180, 180), r"$\phi$ $(^{\circ})$"]
]

In [14]:
fig4, ax4 = plt.subplots(2,4, figsize=(15,10))
ax4=ax4.flatten()

fig4.suptitle('photons (2)', fontsize=16)

for i, distro in enumerate(phot2_distros):
    hist1D(ax4[i], MC_df, distro, cuts=(MC_df.Truth==1))
    
for i, distro in enumerate(phot2_t_distros):
    ax=ax4[i].twinx()

    hist1D(ax, MC_df, distro, cuts=(MC_df.Truth==1), color='brown', lalpha=0.7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Particle Distributions - 'Trimmed'
Truth-Matched Distributions for each final-state particle, with some additional cuts so isolate desired events:

   * 3$\sigma$ $\pi_{0}$-mass cut
   * Neutrons
      * CND only
      * |p| > 350 MeV
   * Photons
      * Require hit in the PCAL

In [7]:
cuts = [
    (MC_df.Pi0flag_cut_3sigPi0IM == 1),
    ((MC_df.Pi0rec_status > 3990) & (MC_df.Pi0rec_status < 4150)),
    (MC_df.Pi0rec_magP >= 0.35),
    (MC_df.hitPCAL == 1),
    (MC_df.Truth==1)
]

cut = ft.reduce(lambda x, y: x & y, cuts[:])


In [13]:
fig5, ax5 = plt.subplots(2,4, figsize=(15,10))
ax5=ax5.flatten()

fig5.suptitle('electrons', fontsize=16)

for i, distro in enumerate(e_distros):
    hist1D(ax5[i], MC_df, distro, cuts=cut, label="Truth-Matched")

for i, distro in enumerate(e_t_distros):
    ax=ax5[i].twinx()

    hist1D(ax, MC_df, distro, cuts=cut, color='brown', lalpha=0.7, label="Generated")
    
lines_1, labels_1 = ax5[0].get_legend_handles_labels()
lines_2, labels_2 = ax.get_legend_handles_labels()


lines = lines_1 + lines_2
labels = labels_1 + labels_2


ax5[3].legend(lines, labels, loc=2)
ax5[4].legend(lines, labels, loc=2)



    
# for i in 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
fig6, ax6 = plt.subplots(2,4, figsize=(15,10))
ax6=ax6.flatten()

fig6.suptitle('neutrons', fontsize=16)

for i, distro in enumerate(neut_distros):
    hist1D(ax6[i], MC_df, distro, cuts=cut, label="Truth-Matched")
    
for i, distro in enumerate(neut_t_distros):
    ax=ax6[i].twinx()

    hist1D(ax, MC_df, distro, cuts=cut, color='brown', lalpha=0.7, label="Generated")
    
lines_1, labels_1 = ax6[0].get_legend_handles_labels()
lines_2, labels_2 = ax.get_legend_handles_labels()


lines = lines_1 + lines_2
labels = labels_1 + labels_2


ax6[3].legend(lines, labels, loc=1)
ax6[4].legend(lines, labels, loc=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
fig7, ax7 = plt.subplots(2,4, figsize=(15,10))
ax7=ax7.flatten()

fig7.suptitle('photons (1)', fontsize=16)

for i, distro in enumerate(phot1_distros):
    hist1D(ax7[i], MC_df, distro, cuts=cut, label="Truth-Matched")
    
for i, distro in enumerate(phot1_t_distros):
    ax=ax7[i].twinx()

    hist1D(ax, MC_df, distro, cuts=cut, color='brown', lalpha=0.7, label="Generated")
    
lines_1, labels_1 = ax7[0].get_legend_handles_labels()
lines_2, labels_2 = ax.get_legend_handles_labels()

lines = lines_1 + lines_2
labels = labels_1 + labels_2


ax7[3].legend(lines, labels, loc=1)
ax7[4].legend(lines, labels, loc=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
fig8, ax8 = plt.subplots(2,4, figsize=(15,10))
ax8=ax8.flatten()

fig8.suptitle('photons (2)', fontsize=16)

for i, distro in enumerate(phot2_distros):
    hist1D(ax8[i], MC_df, distro, cuts=cut, label="Truth-Matched")
    
for i, distro in enumerate(phot2_t_distros):
    ax=ax8[i].twinx()

    hist1D(ax, MC_df, distro, cuts=cut, color='brown', lalpha=0.7, label="Generated")
    
    
lines_1, labels_1 = ax8[0].get_legend_handles_labels()
lines_2, labels_2 = ax.get_legend_handles_labels()

lines = lines_1 + lines_2
labels = labels_1 + labels_2


ax8[3].legend(lines, labels, loc=1)
ax8[4].legend(lines, labels, loc=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Differences

In [20]:
e_diff = [
    ['deltaPi0e_px', (100, -1, 1), r"$\delta p_{x}$"],
    ['deltaPi0e_py', (100, -1, 1), r"$\delta p_{y}$"],
    ['deltaPi0e_pz', (100, -1, 1), r"$\delta p_{z}$"],
    ['deltaPi0e_E', (100, -1, 1), r"$\delta E$"],
    ['deltaPi0e_magP', (100, -1, 1), r"$\delta |p|$"],
    ['deltaPi0e_pT', (100, -1, 1), r"$\delta p_{T}$"],
    ['deltaPi0e_theta', (100, -1, 1), r"$\delta \theta$"],
    ['deltaPi0e_phi', (100, -2, 2), r"$\delta \phi$"]
]

In [21]:
fig9, ax9 = plt.subplots(2,4, figsize=(15,10))
ax9=ax9.flatten()

fig9.suptitle('electron (rec - gen)', fontsize=16)

for i, distro in enumerate(e_diff):
    hist1D(ax9[i], MC_df, distro, cuts=cut, color='orangered', lalpha=0.7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
neut_diff = [
    ['deltaPi0rec_px', (100, -1, 1), r"$\delta p_{x}$"],
    ['deltaPi0rec_py', (100, -1, 1), r"$\delta p_{y}$"],
    ['deltaPi0rec_pz', (100, -1, 1), r"$\delta p_{z}$"],
    ['deltaPi0rec_E', (100, -1, 1), r"$\delta E$"],
    ['deltaPi0rec_magP', (100, -1, 1), r"$\delta |p|$"],
    ['deltaPi0rec_pT', (100, -1, 1), r"$\delta p_{T}$"],
    ['deltaPi0rec_theta', (100, -10, 10), r"$\delta \theta$"],
    ['deltaPi0rec_phi', (100, -20, 20), r"$\delta \phi$"]
]

In [23]:
fig10, ax10 = plt.subplots(2,4, figsize=(15,10))
ax10=ax10.flatten()

fig10.suptitle('neutron (rec - gen)', fontsize=16)

for i, distro in enumerate(neut_diff):
    hist1D(ax10[i], MC_df, distro, cuts=cut, color='orangered', lalpha=0.7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
neut_angles= [
    
    ["deltaPi0rec_theta", (200, -40, 50), r"$\delta\theta$ $(^{\circ})$"],
    ["deltaPi0rec_phi", (200, -60, 60), r"$\delta\phi$ $(^{\circ})$"]
]

fig10b, ax10b = plt.subplots(1,2, figsize=(10,5))
ax10b=ax10b.flatten()

fig10b.suptitle('neutron (rec - gen) - wide range angles', fontsize=16)

for i, distro in enumerate(neut_angles):
    hist1D(ax10b[i], MC_df, distro, cuts=cut, color='orangered', lalpha=0.7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
phot1_diff = [
    ['deltaPi0phot1_px', (100, -1, 1), r"$\delta p_{x}$"],
    ['deltaPi0phot1_py', (100, -1, 1), r"$\delta p_{y}$"],
    ['deltaPi0phot1_pz', (100, -1, 1), r"$\delta p_{z}$"],
    ['deltaPi0phot1_E', (100, -1, 1), r"$\delta E$"],
    ['deltaPi0phot1_magP', (100, -1, 1), r"$\delta |p|$"],
    ['deltaPi0phot1_pT', (100, -1, 1), r"$\delta p_{T}$"],
    ['deltaPi0phot1_theta', (100, -5, 5), r"$\delta \theta$"],
    ['deltaPi0phot1_phi', (100, -5, 5), r"$\delta \phi$"]
]

In [26]:
fig11, ax11 = plt.subplots(2,4, figsize=(15,10))
ax11=ax11.flatten()

fig11.suptitle('photon(1) (rec - gen)', fontsize=16)

for i, distro in enumerate(phot1_diff):
    hist1D(ax11[i], MC_df, distro, cuts=cut, color='orangered', lalpha=0.7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
phot2_diff = [
    ['deltaPi0phot2_px', (100, -1, 1), r"$\delta p_{x}$"],
    ['deltaPi0phot2_py', (100, -1, 1), r"$\delta p_{y}$"],
    ['deltaPi0phot2_pz', (100, -1, 1), r"$\delta p_{z}$"],
    ['deltaPi0phot2_E', (100, -1, 1), r"$\delta E$"],
    ['deltaPi0phot2_magP', (100, -1, 1), r"$\delta |p|$"],
    ['deltaPi0phot2_pT', (100, -1, 1), r"$\delta p_{T}$"],
    ['deltaPi0phot2_theta', (100, -5, 5), r"$\delta \theta$"],
    ['deltaPi0phot2_phi', (100, -5, 5), r"$\delta \phi$"]
]

In [28]:
fig11, ax11 = plt.subplots(2,4, figsize=(15,10))
ax11=ax11.flatten()

fig11.suptitle('photon(2) (rec - gen)', fontsize=16)

for i, distro in enumerate(phot1_diff):
    hist1D(ax11[i], MC_df, distro, cuts=cut, color='orangered', lalpha=0.7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
import sys
sys.path.append("/home/pauln/code/nicks_plot_utils")

In [30]:
from nicks_plot_utils import Hist1D as h1d
from nicks_plot_utils import Hist2D as h2d

In [31]:
plt.style.use('seaborn')
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.formatter.limits'] = (-3,3)
plt.rcParams['axes.formatter.use_mathtext'] = True
plt.rcParams['font.size']= 16
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

In [32]:
neut_diff = [
    ['deltaPi0rec_px', (100, -.5, .5), r"$\delta p_{x}$"],
    ['deltaPi0rec_py', (100, -.5, .5), r"$\delta p_{y}$"],
    ['deltaPi0rec_pz', (100, -.5, .5), r"$\delta p_{z}$"],
    ['deltaPi0rec_E', (100, -.5, .5), r"$\delta E$"],
    ['deltaPi0rec_magP', (100, -.5, .5), r"$\delta |p|$"],
    ['deltaPi0rec_pT', (100, -.5, .5), r"$\delta p_{T}$"],
    ['deltaPi0rec_theta', (100, -7, 7), r"$\delta \theta$"]
]

In [33]:
phi_2Ds = []
# phi_xrange=[-10,50]
# phi_xbins=100

for distro in neut_diff:
    var = distro[0]
    rb = distro[1]
    lbl= distro[2]
    
    h2 = h2d(xrange=[-10,50], xbins=100, yrange=[rb[1],rb[2]], ybins=rb[0], xname=r"$\delta \phi$", yname=lbl)
    h2.fill(MC_df["deltaPi0rec_phi"][cut], MC_df[var][cut])
    
    phi_2Ds.append(h2)



In [34]:
neut_diff = [
    ['deltaPi0rec_px', (100, -.5, .5), r"$\delta p_{x}$"],
    ['deltaPi0rec_py', (100, -.5, .5), r"$\delta p_{y}$"],
    ['deltaPi0rec_pz', (100, -.5, .5), r"$\delta p_{z}$"],
    ['deltaPi0rec_E', (100, -.5, .5), r"$\delta E$"],
    ['deltaPi0rec_magP', (100, -.5, .5), r"$\delta |p|$"],
    ['deltaPi0rec_pT', (100, -.5, .5), r"$\delta p_{T}$"],
    ['deltaPi0rec_phi', (100, -10, 50), r"$\delta \phi$"]
]

In [35]:
theta_2Ds = []
# phi_xrange=[-10,50]
# phi_xbins=100

for distro in neut_diff:
    var = distro[0]
    rb = distro[1]
    lbl= distro[2]
    
    h2 = h2d(xrange=[-15,15], xbins=100, yrange=[rb[1],rb[2]], ybins=rb[0], xname=r"$\delta \theta$", yname=lbl)
    h2.fill(MC_df["deltaPi0rec_theta"][cut], MC_df[var][cut])
    
    theta_2Ds.append(h2)



In [36]:
fig,ax = plt.subplots(7,2, figsize=(15,25))

for i,plot in enumerate(zip(phi_2Ds, theta_2Ds)):
    plot[0].plot(ax=ax[i][0], density=False)
    plot[1].plot(ax=ax[i][1], density=False)
    
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Some follow up plots:

In [37]:
dphi_theta = h2d(xrange=[-10,50], xbins=100, yrange=[0,70], ybins=100, xname=r"$\delta \phi$", yname=r"$\theta$")
dphi_theta.fill(MC_df["deltaPi0rec_phi"][(MC_df.Truth == 1)], MC_df["Pi0rec_theta"][(MC_df.Truth == 1)])

dphi_p = h2d(xrange=[-30,80], xbins=100, yrange=[0,1.75], ybins=100, xname=r"$\delta \phi$", yname=r"$|p|$")
dphi_p.fill(MC_df["deltaPi0rec_phi"][(MC_df.Truth == 1)], MC_df["Pi0rec_magP"][(MC_df.Truth == 1)])

Histogram(
  Regular(100, -30, 80),
  Regular(100, 0, 1.75),
  storage=Double()) # Sum: 20225.0 (22939.0 with flow)

In [38]:
dphi_theta_cut = h2d(xrange=[-10,50], xbins=100, yrange=[0,70], ybins=100, xname=r"$\delta \phi (cut)$", yname=r"$\theta (cut)$")
dphi_theta_cut.fill(MC_df["deltaPi0rec_phi"][cut], MC_df["Pi0rec_theta"][cut])

dphi_p_cut = h2d(xrange=[-30,80], xbins=100, yrange=[0,1.75], ybins=100, xname=r"$\delta \phi$", yname=r"$|p|$")
dphi_p_cut.fill(MC_df["deltaPi0rec_phi"][cut], MC_df["Pi0rec_magP"][cut])

Histogram(
  Regular(100, -30, 80),
  Regular(100, 0, 1.75),
  storage=Double()) # Sum: 13960.0 (15014.0 with flow)

In [39]:
n_1D_Phi = h1d(data=MC_df["deltaPi0rec_phi"][cut], xrange=[-30, 80], bins=100)


In [40]:
fig, ax = plt.subplots(2,2, figsize=(17,15))
ax=ax.flatten()

dphi_theta.plot(ax=ax[0], density=False)
dphi_p.plot(ax=ax[1], density=False)
dphi_theta_cut.plot(ax=ax[2], density=False)
#n_1D_Phi.histogram(ax=ax[3], density=True, color="brown")
dphi_p_cut.plot(ax=ax[3], density=False)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
fig, ax = plt.subplots(1, 3, figsize=(15,5))

hist1D(ax[0], MC_df, ["truPi0rec_phi", (130, -180, 180), r"$neutron$ $\phi$ - [all truth]"], color='brown', lalpha=.7)

axb=ax[1].twinx()
hist1D(ax[1], MC_df, ["Pi0rec_phi", (48, -180, 180), r"$neutron$ $\phi$"], cuts = cut&(MC_df.Truth == 1), label="Truth-Matched")
hist1D(axb, MC_df, ["truPi0rec_phi", (48, -180, 180), r"$neutron$ $\phi$"], cuts = cut&(MC_df.Truth == 1), color='brown', lalpha=.7, label="Generated")

hist1D(ax[2], MC_df, ["Pi0rec_phi", (48, -180, 180), r"$neutron$ $\phi$"], cuts = cut&(MC_df.Truth == 1)&(MC_df.Pi0flag_MC_neutrec==1), label="Truth-Matched + PID")
axc=ax[2].twinx()
hist1D(axc, MC_df, ["truPi0rec_phi", (48, -180, 180), r"$neutron$ $\phi$"], cuts = cut&(MC_df.Truth == 1)&(MC_df.Pi0flag_MC_neutrec==1), color='brown', lalpha=.7, label="Generated")


lines_1, labels_1 = ax[1].get_legend_handles_labels()
lines_2, labels_2 = axb.get_legend_handles_labels()

lines = lines_1 + lines_2
labels = labels_1 + labels_2


ax[1].legend(lines, labels, loc=3)

lines_1, labels_1 = ax[2].get_legend_handles_labels()
lines_2, labels_2 = axc.get_legend_handles_labels()

lines = lines_1 + lines_2
labels = labels_1 + labels_2


ax[2].legend(lines, labels, loc=3)
#def hist1D(ax, df, var, bins = None, cuts=None, y_err=False, color=None, lalpha=1, falpha=0.4, normed=False, filled = False, density = False, params={}):


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
dphi_truPhi = h2d(
    ydata=MC_df["deltaPi0rec_phi"][(MC_df.Truth == 1)],
    xdata=MC_df["truPi0rec_phi"][(MC_df.Truth == 1)],
    yrange=[-10,50], 
    ybins=100, 
    xrange=[-180,180], 
    xbins=48, 
    yname=r"$\delta \phi (cut)$", 
    xname=r"$true\ \phi$"
)

dphi_Phi = h2d(
    ydata=MC_df["deltaPi0rec_phi"][cut&(MC_df.Truth == 1)],
    xdata=MC_df["Pi0rec_phi"][cut&(MC_df.Truth == 1)],
    yrange=[-10,50], 
    ybins=100, 
    xrange=[-180,180], 
    xbins=48, 
    yname=r"$\delta \phi$", 
    xname=r"$\phi$"
)

# n_1D_Phi = h1d(data=MC_df["Pi0rec_phi"][(MC_df.Truth == 1)],
#                xrange=[-180, 180],
#                bins=48)
# n_1D_truPhi = h1d(data=MC_df["truPi0rec_phi"][(MC_df.Truth == 1)],
#                xrange=[-180, 180],
#                bins=48)

In [43]:
fig,ax = plt.subplots(2,2, figsize=(12,10))
ax=ax.flatten()

hist1D(ax[0], MC_df, ["truPi0rec_phi", (48, -180, 180), r"$neutron$ $\phi$ - [truth  & True==1]"], cuts = cut&(MC_df.Truth == 1), color='brown', lalpha=.7)
hist1D(ax[1], MC_df, ["Pi0rec_phi", (48, -180, 180), r"$neutron$ $\phi$ -  [truth & True==1]"], cuts = cut&(MC_df.Truth == 1))

dphi_truPhi.plot(ax=ax[2], density=False, colorbar=False)
dphi_Phi.plot(ax=ax[3], density=False, colorbar=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
fig,ax = plt.subplots(1,2, figsize=(12,6))


dphi_truPhi.plot(ax=ax[0], density=False, colorbar=False)
ax0=ax[0].twinx()
ax0.grid(False)
hist1D(ax0, MC_df, ["truPi0rec_phi", (48, -180, 180), r"$neutron$ $\phi$ - [truth  & True==1]"], cuts = (MC_df.Truth == 1), color='brown', lalpha=.7)


dphi_Phi.plot(ax=ax[1], density=False, colorbar=False)
ax1=ax[1].twinx()
ax1.grid(False)
hist1D(ax1, MC_df, ["Pi0rec_phi", (48, -180, 180), r"$neutron$ $\phi$ -  [truth & True==1]"], cuts = (MC_df.Truth == 1), color="blue", lalpha=.5)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
dphi_truPhi = h2d(
    ydata=MC_df["deltaPi0rec_phi"][(MC_df.Truth == 1)],
    xdata=MC_df["truPi0rec_phi"][(MC_df.Truth == 1)],
    yrange=[-9,9], 
    ybins=10, 
    xrange=[-180,180], 
    xbins=48, 
    yname=r"$\delta \phi (cut)$", 
    xname=r"$true\ \phi$"
)

dphi_Phi = h2d(
    ydata=MC_df["deltaPi0rec_phi"][(MC_df.Truth == 1)],
    xdata=MC_df["Pi0rec_phi"][(MC_df.Truth == 1)],
    yrange=[-9,9], 
    ybins=10, 
    xrange=[-180,180], 
    xbins=48, 
    yname=r"$\delta \phi (cut)$", 
    xname=r"$truth-matched\ \phi$"
)



In [46]:
fig,ax = plt.subplots(1,2, figsize=(13,5))
dphi_truPhi.plot(ax=ax[0], density=False)
dphi_Phi.plot(ax=ax[1], density=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Status Bank for the neutron
Parameterises where in the detector suite an event has been detected.
   * 4000 => CD hit
   * \+ 100 * N CTOF responses
   * \+ 10 * N CND responses 
   
---

We have: 
   * mostly CND only hits (4010)
   * some CTOF only hits (4100)
   * a few CTOF and CND events (4110)

In [107]:
fig,ax = plt.subplots()
hist1D(ax, MC_df, ["Pi0rec_status", (2000, 3950, 4150), r"$neutron$ $status$"], color='brown', lalpha=.7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [108]:
neut_diff = [
    ['deltaPi0rec_px', (100, -1, 1), r"$\delta p_{x}$"],
    ['deltaPi0rec_py', (100, -1, 1), r"$\delta p_{y}$"],
    ['deltaPi0rec_pz', (100, -1, 1), r"$\delta p_{z}$"],
    ['deltaPi0rec_E', (100, -1, 1), r"$\delta E$"],
    ['deltaPi0rec_magP', (100, -1, 1), r"$\delta |p|$"],
    ['deltaPi0rec_pT', (100, -1, 1), r"$\delta p_{T}$"],
    ['deltaPi0rec_theta', (100, -10, 10), r"$\delta \theta$"],
    ['deltaPi0rec_phi', (100, -20, 60), r"$\delta \phi$"]
]

In [109]:
fig10, ax10 = plt.subplots(2,4, figsize=(15,10))
ax10=ax10.flatten()

fig10.suptitle('neutron (rec - gen)', fontsize=16)

for i, distro in enumerate(neut_diff):
    hist1D(ax10[i], MC_df, distro, cuts=cut&(MC_df.Pi0rec_status==4010), label="CND", bins=50)
    hist1D(ax10[i], MC_df, distro, cuts=cut&(MC_df.Pi0rec_status==4100), label="CTOF", bins=50)
    hist1D(ax10[i], MC_df, distro, cuts=cut&(MC_df.Pi0rec_status==4110), label="CND/CTOF", bins=50)
    
ax10[3].legend(loc=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [110]:
fig10, ax10 = plt.subplots(2,4, figsize=(15,10))
ax10=ax10.flatten()

fig10.suptitle('neutron (rec - gen)', fontsize=16)

for i, distro in enumerate(neut_diff):
    hist1D(ax10[i], MC_df, distro, cuts=cut&(MC_df.Pi0rec_status==4010), density=True, label="CND", bins=50)
    hist1D(ax10[i], MC_df, distro, cuts=cut&(MC_df.Pi0rec_status==4100), density=True, label="CTOF", bins=50)
    hist1D(ax10[i], MC_df, distro, cuts=cut&(MC_df.Pi0rec_status==4110), density=True, label="CND/CTOF", bins=50)
    
ax10[3].legend(loc=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [111]:
dphi_p_stat1 = h2d(
    xdata=MC_df["deltaPi0rec_phi"][(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4010)], 
    ydata=MC_df["Pi0rec_magP"][(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4010)], 
    xrange=[-30,80], 
    xbins=100, 
    yrange=[0,1.75], 
    ybins=100, 
    xname=r"$\delta \phi (cut)$", 
    yname=r"$|p| (cut)$"
)

dphi_p_stat2 = h2d(
    xdata=MC_df["deltaPi0rec_phi"][(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4100)], 
    ydata=MC_df["Pi0rec_magP"][(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4100)], 
    xrange=[-30,80], 
    xbins=70, 
    yrange=[0,1.75], 
    ybins=70, 
    xname=r"$\delta \phi (cut)$", 
    yname=r"$|p| (cut)$"
)

dphi_p_stat3 = h2d(
    xdata=MC_df["deltaPi0rec_phi"][(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4110)], 
    ydata=MC_df["Pi0rec_magP"][(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4110)], 
    xrange=[-30,80], 
    xbins=30, 
    yrange=[0,1.75], 
    ybins=30, 
    xname=r"$\delta \phi (cut)$", 
    yname=r"$|p| (cut)$"
)


In [112]:
fig,ax = plt.subplots(1,3, figsize=(15,5))

dphi_p_stat1.plot(ax=ax[0], density=False)
dphi_p_stat2.plot(ax=ax[1], density=False)
dphi_p_stat3.plot(ax=ax[2], density=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [151]:
dphi_phi_stat1 = h2d(
    ydata=MC_df["deltaPi0rec_phi"][cut&(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4010)], 
    xdata=MC_df["Pi0rec_phi"][cut&(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4010)], 
    yrange=[-10,50], 
    ybins=80, 
    xrange=[-180,180], 
    xbins=48, 
    yname=r"$\delta \phi$", 
    xname=r"$\phi$"
)

dphi_phi_stat2 = h2d(
    ydata=MC_df["deltaPi0rec_phi"][cut&(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4100)], 
    xdata=MC_df["Pi0rec_phi"][cut&(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4100)], 
    yrange=[-10,50], 
    ybins=60, 
    xrange=[-180,180], 
    xbins=48, 
    yname=r"$\delta \phi$", 
    xname=r"$\phi$"
)

dphi_phi_stat3 = h2d(
    ydata=MC_df["deltaPi0rec_phi"][cut&(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4110)], 
    xdata=MC_df["Pi0rec_phi"][cut&(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4110)], 
    yrange=[-10,50], 
    ybins=20, 
    xrange=[-180,180], 
    xbins=48,  
    yname=r"$\delta \phi$", 
    xname=r"$\phi$"
)


In [152]:
fig,ax = plt.subplots(1,3, figsiWhat do you see on the MC that needs to be understood?ze=(15,5))

dphi_phi_stat1.plot(ax=ax[0], density=False)
dphi_phi_stat2.plot(ax=ax[1], density=False)
dphi_phi_stat3.plot(ax=ax[2], density=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [149]:
dphi_phi_stat1 = h2d(
    ydata=MC_df["deltaPi0rec_phi"][(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4010)&cut], 
    xdata=MC_df["Pi0rec_phi"][(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4010)&cut], 
    yrange=[-10,50], 
    ybins=80, 
    xrange=[-180,180], 
    xbins=48, 
    yname=r"$\delta \phi (cut)$", 
    xname=r"$\phi (cut)$"
)

dphi_phi_stat2 = h2d(
    ydata=MC_df["deltaPi0rec_phi"][(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4100)&cut], 
    xdata=MC_df["Pi0rec_phi"][(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4100)&cut], 
    yrange=[-10,50], 
    ybins=60, 
    xrange=[-180,180], 
    xbins=48, 
    yname=r"$\delta \phi (cut)$", 
    xname=r"$\phi (cut)$"
)

dphi_phi_stat3 = h2d(
    ydata=MC_df["deltaPi0rec_phi"][(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4110)&cut], 
    xdata=MC_df["Pi0rec_phi"][(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4110)&cut], 
    yrange=[-10,50], 
    ybins=20, 
    xrange=[-180,180], 
    xbins=48,  
    yname=r"$\delta \phi (cut)$", 
    xname=r"$\phi (cut)$"
)


In [187]:
fig,ax = plt.subplots(1,3, figsize=(15,5))

dphi_phi_stat1.plot(ax=ax[0], density=False)
dphi_phi_stat2.plot(ax=ax[1], density=False)
dphi_phi_stat3.plot(ax=ax[2], density=False)

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [115]:
dphi_Phi_stat1 = h2d(
    ydata=MC_df["deltaPi0rec_phi"][(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4010)],
    xdata=MC_df["Pi0rec_phi"][(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4010)],
    yrange=[-9,9], 
    ybins=4, 
    xrange=[-180,180], 
    xbins=48, 
    yname=r"$\delta \phi (cut)$", 
    xname=r"$truth-matched\ \phi$"
)

dphi_Phi_stat2 = h2d(
    ydata=MC_df["deltaPi0rec_phi"][(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4100)],
    xdata=MC_df["Pi0rec_phi"][(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4100)],
    yrange=[-9,9], 
    ybins=4, 
    xrange=[-180,180], 
    xbins=48, 
    yname=r"$\delta \phi (cut)$", 
    xname=r"$truth-matched\ \phi$"
)

dphi_Phi_stat3 = h2d(
    ydata=MC_df["deltaPi0rec_phi"][(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4110)],
    xdata=MC_df["Pi0rec_phi"][(MC_df.Truth == 1)&(MC_df.Pi0rec_status==4110)],
    yrange=[-9,9], 
    ybins=4, 
    xrange=[-180,180], 
    xbins=48, 
    yname=r"$\delta \phi (cut)$", 
    xname=r"$truth-matched\ \phi$"
)



In [116]:
fig,ax = plt.subplots(1,3, figsize=(18,5))

dphi_Phi_stat1.plot(ax=ax[0], density=False)
dphi_Phi_stat2.plot(ax=ax[1], density=False)
dphi_Phi_stat3.plot(ax=ax[2], density=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Adding PID matching on truth

In [117]:
fig2, ax2 = plt.subplots(2,4, figsize=(15,10))
ax2=ax2.flatten()

fig2.suptitle('neutrons', fontsize=16)

for i, distro in enumerate(neut_distros):
    hist1D(ax2[i], MC_df, distro, cuts=(MC_df.Truth==1)&(MC_df.Pi0flag_MC_neutrec==1))
    
for i, distro in enumerate(neut_t_distros):
    ax=ax2[i].twinx()

    hist1D(ax, MC_df, distro, cuts=(MC_df.Truth==1)&(MC_df.Pi0flag_MC_neutrec==1), color='brown', lalpha=0.7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [168]:
fig6, ax6 = plt.subplots(2,4, figsize=(15,10))
ax6=ax6.flatten()

fig6.suptitle('neutrons', fontsize=16)

for i, distro in enumerate(neut_distros):
    hist1D(ax6[i], MC_df, distro, cuts=cut&(MC_df.Pi0flag_MC_neutrec==1), label="Truth & PID")
    
for i, distro in enumerate(neut_t_distros):
    ax=ax6[i].twinx()

    hist1D(ax, MC_df, distro, cuts=cut&(MC_df.Pi0flag_MC_neutrec==1), color='brown', lalpha=0.7, label="Generated")
    
lines_1, labels_1 = ax6[0].get_legend_handles_labels()
lines_2, labels_2 = ax.get_legend_handles_labels()

lines = lines_1 + lines_2
labels = labels_1 + labels_2


ax6[3].legend(lines, labels, loc=1)
ax6[4].legend(lines, labels, loc=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [119]:
neut_diff = [
    ['deltaPi0rec_px', (100, -.5, .5), r"$\delta p_{x}$"],
    ['deltaPi0rec_py', (100, -.5, .5), r"$\delta p_{y}$"],
    ['deltaPi0rec_pz', (100, -.5, .5), r"$\delta p_{z}$"],
    ['deltaPi0rec_E', (100, -.5, .5), r"$\delta E$"],
    ['deltaPi0rec_magP', (100, -.5, .5), r"$\delta |p|$"],
    ['deltaPi0rec_pT', (100, -.5, .5), r"$\delta p_{T}$"],
    ['deltaPi0rec_theta', (100, -10, 10), r"$\delta \theta$"],    
    ['deltaPi0rec_phi', (100, -10, 50), r"$\delta \phi$"]
]

In [120]:
fig10, ax10 = plt.subplots(2,4, figsize=(15,10))
ax10=ax10.flatten()

fig10.suptitle('neutron (rec - gen)', fontsize=16)

for i, distro in enumerate(neut_diff):
    hist1D(ax10[i], MC_df, distro, cuts=cut&(MC_df.Pi0flag_MC_neutrec==1), color='orangered', lalpha=0.7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [121]:
dphi_p_cut = h2d(xrange=[-30,80], xbins=100, yrange=[0,1.75], ybins=100, xname=r"$\delta \phi (cut)$", yname=r"$|p| (cut)$")
dphi_p_cut.fill(MC_df["deltaPi0rec_phi"][cut&(MC_df.Pi0flag_MC_neutrec==1)], MC_df["Pi0rec_magP"][cut&(MC_df.Pi0flag_MC_neutrec==1)])

Histogram(
  Regular(100, -30, 80),
  Regular(100, 0, 1.75),
  storage=Double()) # Sum: 5920.0 (6040.0 with flow)

In [122]:
fig,ax = plt.subplots()
dphi_p_cut.plot(ax=ax, density=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Status Bank with PID added

In [146]:
fig,ax = plt.subplots()
hist1D(ax, MC_df, ["Pi0rec_status", (2000, 3950, 4150), r"$neutron$ $status$"], cuts=(MC_df.Truth==1), color='black', label="Truth-Matched")
ax2=ax.twinx()
hist1D(ax2, MC_df, ["Pi0rec_status", (2000, 3950, 4150), r"$neutron$ $status$"], cuts=(MC_df.Truth==1)&(MC_df.Pi0flag_MC_neutrec==1), color='red', lalpha=.5, label="Truth-Matched with PID")

ax.legend(loc=2)
ax2.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [140]:
neut_diff = [
    ['deltaPi0rec_px', (100, -1, 1), r"$\delta p_{x}$"],
    ['deltaPi0rec_py', (100, -1, 1), r"$\delta p_{y}$"],
    ['deltaPi0rec_pz', (100, -1, 1), r"$\delta p_{z}$"],
    ['deltaPi0rec_E', (100, -1, 1), r"$\delta E$"],
    ['deltaPi0rec_magP', (100, -1, 1), r"$\delta |p|$"],
    ['deltaPi0rec_pT', (100, -1, 1), r"$\delta p_{T}$"],
    ['deltaPi0rec_theta', (100, -10, 10), r"$\delta \theta$"],
    ['deltaPi0rec_phi', (100, -20, 60), r"$\delta \phi$"]
]

In [135]:
fig10, ax10 = plt.subplots(2,4, figsize=(15,10))
ax10=ax10.flatten()

fig10.suptitle('neutron (rec - gen) - with PUID', fontsize=16)

for i, distro in enumerate(neut_diff):
    hist1D(ax10[i], MC_df, distro, cuts=cut&(MC_df.Pi0rec_status==4010)&(MC_df.Pi0flag_MC_neutrec==1), label="CND", bins=50)
    hist1D(ax10[i], MC_df, distro, cuts=cut&(MC_df.Pi0rec_status==4100)&(MC_df.Pi0flag_MC_neutrec==1), label="CTOF", bins=50)
    hist1D(ax10[i], MC_df, distro, cuts=cut&(MC_df.Pi0rec_status==4110)&(MC_df.Pi0flag_MC_neutrec==1), label="CND/CTOF", bins=50)
    
ax10[3].legend(loc=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [194]:
fig10, ax10 = plt.subplots(2,4, figsize=(15,10))
ax10=ax10.flatten()

fig10.suptitle('neutron (rec - gen) - with PID', fontsize=16)

for i, distro in enumerate(neut_diff):
    hist1D(ax10[i], MC_df, distro, cuts=cut&(MC_df.Pi0rec_status==4010)&(MC_df.Pi0flag_MC_neutrec==1), density=True, label="CND", bins=50)
    hist1D(ax10[i], MC_df, distro, cuts=cut&(MC_df.Pi0rec_status==4100)&(MC_df.Pi0flag_MC_neutrec==1), density=True, label="CTOF", bins=50)
    hist1D(ax10[i], MC_df, distro, cuts=cut&(MC_df.Pi0rec_status==4110)&(MC_df.Pi0flag_MC_neutrec==1), density=True, label="CND/CTOF", bins=25)
    
ax10[3].legend(loc=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# P/$\theta$ for efficiency calc

In [10]:
bins=150
distros = [
    ["Pi0e_magP", (bins, 0, 9), r"$|p|$ $(GeV/c^{2})$"],
    ["Pi0e_theta", (bins, 0, 40), r"$\theta$ $(^{\circ})$"],
    ["Pi0phot1_magP", (bins, 0, 9), r"$|p|$ $(GeV/c^{2})$"],
    ["Pi0phot1_theta", (bins, 0, 40), r"$\theta$ $(^{\circ})$"],
    ["Pi0phot2_magP", (bins, 0, 5), r"$|p|$ $(GeV/c^{2})$"],
    ["Pi0phot2_theta", (bins, 0, 40), r"$\theta$ $(^{\circ})$"]
]

tdistros = [
    ["truPi0e_magP", (bins, 0, 9), r"$|p|$ $(GeV/c^{2})$"],
    ["truPi0e_theta", (bins, 0, 40), r"$\theta$ $(^{\circ})$"],
    ["truPi0phot1_magP", (bins, 0, 9), r"$|p|$ $(GeV/c^{2})$"],
    ["truPi0phot1_theta", (bins, 0, 40), r"$\theta$ $(^{\circ})$"],
    ["truPi0phot2_magP", (bins, 0, 5), r"$|p|$ $(GeV/c^{2})$"],
    ["truPi0phot2_theta", (bins, 0, 40), r"$\theta$ $(^{\circ})$"]
]


In [20]:
fig, ax = plt.subplots(3,2,figsize=(12,20))
ax = ax.flatten()

for i, distro in enumerate(distros):
    hist1D(ax[i], MC_df, distro, cuts=(MC_df.Truth==1), label="Recon.")
    
for i, distro in enumerate(tdistros):
    hist1D(ax[i], MC_df, distro, cuts=(MC_df.Truth==1), color="brown", lalpha=0.6, label="Gen.")
    ax[i].legend(loc=0)
ax[0].legend(loc=2)

ax[0].set_ylabel("electrons")
ax[2].set_ylabel("photons (1)")
ax[4].set_ylabel("photons (2)")


plt.tight_layout(pad=0.3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
neut_distros =[
    ["Pi0rec_magP", (bins, 0, 2), r"$|p|$ $(GeV/c^{2})$"],
    ["Pi0rec_theta", (bins, 0, 180), r"$\theta$ $(^{\circ})$"]
]
neut_tdistros =[
    ["truPi0rec_magP", (bins, 0, 2), r"$|p|$ $(GeV/c^{2})$"],
    ["truPi0rec_theta", (bins, 0, 180), r"$\theta$ $(^{\circ})$"]
]

# neut_cuts = [
#     (MC_df.Pi0flag_cut_3sigPi0IM == 1),
#     ((MC_df.Pi0rec_status > 3990) & (MC_df.Pi0rec_status < 4150)),
#     (MC_df.Pi0rec_magP >= 0.35),
#     (MC_df.hitPCAL == 1),
#     (MC_df.Truth==1)
# ]

# cut = ft.reduce(lambda x, y: x & y, cuts[:])



In [19]:
fig, ax = plt.subplots(1,2,figsize=(12,6.3))
ax = ax.flatten()

for i, distro in enumerate(neut_distros):
    hist1D(ax[i], MC_df, distro, cuts=(MC_df.Truth==1)&((MC_df.Pi0rec_status > 3990) & (MC_df.Pi0rec_status < 4150)), label="Recon.")
    
for i, distro in enumerate(neut_tdistros):
    hist1D(ax[i], MC_df, distro, cuts=(MC_df.Truth==1)&((MC_df.Pi0rec_status > 3990) & (MC_df.Pi0rec_status < 4150)), color="brown", lalpha=0.6, label="Gen.")
    ax[i].legend(loc=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
MC_root_file = "/w/work3/home/pauln/analysis/processed/chanser_MC15/pauln/Pi0_PID_fullcomb/FinalState.root"

tree= "FINALOUTTREE"

data = uproot.open(MC_root_file)[tree]
MC15_df = data.pandas.df()

KeyboardInterrupt: 

In [46]:
list(MC15_df.columns)

['Pi0helicity',
 'Pi0W2',
 'Pi0Q2',
 'Pi0xB',
 'Pi0tneg',
 'Pi0tneg_pi0',
 'Pi0dtneg',
 'Pi0IM_g1g2',
 'Pi0MM2_total',
 'Pi0MP_total',
 'Pi0ME_total',
 'Pi0MPt_total',
 'Pi0pi0coneangle',
 'Pi0eg1coneangle',
 'Pi0eg2coneangle',
 'Pi0recoilconeangle',
 'Pi0MP_rec_recoil',
 'Pi0MPt_rec_recoil',
 'Pi0MM_rec_recoil',
 'Pi0MM2_rec_recoil',
 'Pi0MP_rec_spectator',
 'Pi0MPt_rec_spectator',
 'Pi0MM_rec_spectator',
 'Pi0MM2_rec_spectator',
 'Pi0phi_Nvg',
 'Pi0phi_Nnew',
 'Pi0phi_vgnew',
 'Pi0cop_Nvg_vgnew',
 'Pi0cop_Nvg_Nnew',
 'Pi0cop_Nnew_vgnew',
 'Pi0recon_recoil_T',
 'Pi0recoil_T',
 'Pi0dneutT',
 'Pi0rec_px',
 'Pi0rec_py',
 'Pi0rec_pz',
 'Pi0rec_E',
 'Pi0rec_magP',
 'Pi0rec_pT',
 'Pi0rec_theta',
 'Pi0rec_phi',
 'Pi0rec_status',
 'Pi0rec_PID',
 'Pi0e_px',
 'Pi0e_py',
 'Pi0e_pz',
 'Pi0e_E',
 'Pi0e_magP',
 'Pi0e_pT',
 'Pi0e_theta',
 'Pi0e_phi',
 'Pi0e_status',
 'Pi0phot1_px',
 'Pi0phot1_py',
 'Pi0phot1_pz',
 'Pi0phot1_E',
 'Pi0phot1_magP',
 'Pi0phot1_pT',
 'Pi0phot1_theta',
 'Pi0phot1_phi',
 '

In [202]:
len(MC15_df['truPi0flag_MC_neutrec'][MC15_df.Truth==1])

64164

In [203]:
len(MC15_df['truPi0flag_MC_neutrec'][(MC15_df.Truth==1)&(MC15_df.Pi0flag_MC_neutrec==1)])

9865

In [205]:
len(MC15_df['truPi0flag_MC_neutrec'][(MC15_df.Truth==1)&(MC15_df.Pi0flag_MC_neutrec==0)])

54299

In [204]:
len(MC15_df['truPi0flag_MC_neutrec'])

986841

In [206]:
len(MC15_df['truPi0flag_MC_neutrec'][(MC15_df.Truth==1)&(MC_df.Pi0rec_status > 3990) & (MC_df.Pi0rec_status < 4150)])

41218

In [207]:
len(MC15_df['truPi0flag_MC_neutrec'][(MC15_df.Truth==1)&(MC15_df.Pi0flag_MC_neutrec==1)&(MC_df.Pi0rec_status > 3990) & (MC_df.Pi0rec_status < 4150)])

6399

In [208]:
len(MC15_df['truPi0flag_MC_neutrec'][(MC15_df.Truth==1)&(MC15_df.Pi0flag_MC_neutrec==0)&(MC_df.Pi0rec_status > 3990) & (MC_df.Pi0rec_status < 4150)])

34819

Is PCAL hit flag culling FT photons?

In [50]:
photon1_theta1 = h1d(data=MC_df["Pi0phot1_theta"][(MC_df.Truth == 1)],#&(MC_df.Pi0flag_MC_neutrec==0)],
               xrange=[0, 70],
               bins=200)

photon1_theta2 = h1d(data=MC_df["Pi0phot1_theta"][(MC_df.Truth == 1)&(MC_df.hitPCAL == 1)],#&(MC_df.Pi0flag_MC_neutrec==0)],
               xrange=[0, 70],
               bins=200)

In [51]:
fig, ax = plt.subplots(figsize=(5,5))

photon1_theta1.histogram(ax=ax, label="truth only", density=False)
photon1_theta2.histogram(ax=ax, label="hitPCAL flag", density=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …